In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.0 MB/s 
     |████████████████████████████████| 981 kB 19.8 MB/s 
     |████████████████████████████████| 358 kB 14.4 MB/s 
     |████████████████████████████████| 139 kB 52.4 MB/s 
     |████████████████████████████████| 4.1 MB 26.5 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: urll

In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
            unattached_dfs.append(process(pd.read_csv(address)))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     unattached_dfs = []
     data = np.concatenate(nud)
     unattached_dfs = []
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [24]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd"]
download_data(symbols,"730d","1h")

Symbols : 1500
Data Folder Removed
 
Files In Data : 6


In [31]:
clmns = list(range(1, 169))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1,1))
    row = scaler.fit_transform(row)
    return row
def make_y(data,i,mean_candle):
  current_price = (data[i][0] + data[i][3])/2
  delta = mean_candle * 4
  high =round(current_price + delta)
  low = round(current_price - delta)
  sugg = "sell"
  for p in range(i+1,i+15):
          if data[p][3] <= low:
            sugg = "sell"
            break
          if data[p][3] >= high:
            sugg = "buy"
            break
  return sugg
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    lst = []
    for i in range(11,101,10):
            lst2 = []
            for x in range(i,i-10,-1):
              lst2.append(abs(data[-i][3]-data[-i][0]))
            lst.append(sum(lst2)/len(lst2))
    mean_candle = sum(lst)/len(lst)

    for i in range(85, data.shape[0]-15):
        grow = []
        ggggrow = []
        for x in range(1, 85):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()
        sugg = make_y(data,i,mean_candle)
        arr = np.concatenate((arr, arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [32]:
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Processing File:
1 2 3 4 5 6  
(102480, 170)
(92232, 168) (92232, 2)
(10248, 168) (10248, 2)


In [34]:
from keras.regularizers import l2


In [37]:
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D


In [38]:
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(900,activation="relu", kernel_regularizer=l2(0.00005), bias_regularizer=l2(0.00005)))
model.add(Dropout(0.1))
model.add(Dense(900,activation="relu", kernel_regularizer=l2(0.00005), bias_regularizer=l2(0.00005)))
model.add(Dropout(0.1))
model.add(Dense(900,activation="relu", kernel_regularizer=l2(0.00005), bias_regularizer=l2(0.00005)))
model.add(Dropout(0.1))
model.add(Dense(900,activation="relu", kernel_regularizer=l2(0.00005), bias_regularizer=l2(0.00005)))

model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax(learning_rate = 0.001)

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 1024)              173056    
                                                                 
 dense_40 (Dense)            (None, 900)               922500    
                                                                 
 dropout (Dropout)           (None, 900)               0         
                                                                 
 dense_41 (Dense)            (None, 900)               810900    
                                                                 
 dropout_1 (Dropout)         (None, 900)               0         
                                                                 
 dense_42 (Dense)            (None, 900)               810900    
                                                                 
 dropout_2 (Dropout)         (None, 900)              

In [39]:
model.fit(xTrain,yTrain,epochs=20,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"DCG136_{acr}.h5")

Epoch 1/20
2883/2883 [==============================] - 161s 56ms/step - loss: 0.7497 - accuracy: 0.5510 - val_loss: 0.6953 - val_accuracy: 0.5464
Epoch 2/20
2883/2883 [==============================] - 153s 53ms/step - loss: 0.6888 - accuracy: 0.5522 - val_loss: 0.6889 - val_accuracy: 0.5464
Epoch 3/20
2883/2883 [==============================] - 158s 55ms/step - loss: 0.6878 - accuracy: 0.5522 - val_loss: 0.6889 - val_accuracy: 0.5464
Epoch 4/20
2883/2883 [==============================] - 156s 54ms/step - loss: 0.6878 - accuracy: 0.5522 - val_loss: 0.6888 - val_accuracy: 0.5464
Epoch 5/20
2883/2883 [==============================] - 151s 52ms/step - loss: 0.6878 - accuracy: 0.5522 - val_loss: 0.6890 - val_accuracy: 0.5464
Epoch 6/20
2883/2883 [==============================] - 151s 53ms/step - loss: 0.6878 - accuracy: 0.5522 - val_loss: 0.6890 - val_accuracy: 0.5464
Epoch 7/20
2883/2883 [==============================] - 155s 54ms/step - loss: 0.6877 - accuracy: 0.5522 - val_loss: 0

In [ ]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)
        grow = []
        ggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()
        sugg = make_y(data,i)
        arr = np.concatenate((arr, arr4), axis=0).reshape(-1,1)
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe),index)
    return f"YF  : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
   tv = TvDatafeed()
   raw_data = process_for_prediction(tv.get_hist(symbol=symbol,exchange=exchange,interval=timeframe,n_bars=100),tindex)
   return  f"TVB : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [ ]:
make_prediction("trx-usd","max","1d","trxusdt","binance",Interval.in_daily,-1,-1)

In [ ]:
yf.download("btc-usd",period="max",interval="1mo")